In [2]:
import pandas as pd
from pathlib import Path
import os
import numpy as np


In [3]:
#Load in ForcedAligner output
audio_onsets = pd.read_csv('/home/neel/Desktop/MOUS_hierarchical-representations/MOUS_audio_onset_offsets_with_duration.csv')



In [4]:
#filenames
audio_onsets['TextGrid'] = audio_onsets['TextGrid'].apply(lambda x: x.split('/home/at/workdir/example_workbook.tg/EQ_Ramp_Int2_Int1LPF')[1].split('.TextGrid')[0])
audio_onsets


,TextGrid,Transcription,AlignOnset,AlignOffset,WordCount,NumWords,OnsetPhoneme,OffsetPhoneme,PresegBegin,Duration
0,001,Toen,0.00,0.25,1,13,t,n,0,0.25
1,001,de,0.25,0.40,2,13,d,@,0,0.15
2,001,barkeeper,0.40,1.19,3,13,b,r,0,0.79
3,001,die,1.19,1.35,4,13,d,i,0,0.16
4,001,de,1.35,1.42,5,13,d,@,0,0.07
...,...,...,...,...,...,...,...,...,...,...
9116,908,zorgen,4.21,4.72,6,10,z,@,0,0.51
9117,908,plezier,4.83,5.38,7,10,p,r,0,0.55
9118,908,blije,5.62,6.17,8,10,b,@,0,0.55
9119,908,intens,6.32,6.86,9,10,I,s,0,0.54


In [5]:
events_path = '/media/neel/MOUS/MOUS/MOUS/SynologyDrive/source/sub-A2002/func/sub-A2002_task-auditory_events.tsv'
events = pd.read_csv(events_path, sep='\t')
events

,onset,duration,volume,type,value
0,0.000,NaN,1.000,Pulse,50
1,2.000,NaN,2.000,Pulse,50
2,4.000,NaN,3.000,Pulse,50
3,4.306,NaN,3.153,Picture,WOORDEN
4,6.000,NaN,4.000,Pulse,50
...,...,...,...,...,...
1429,1279.456,NaN,640.728,Nothing,5 Audio onset
1430,1279.980,NaN,640.990,Nothing,6 Target
1431,1280.000,NaN,641.000,Pulse,50
1432,1282.000,NaN,642.000,Pulse,50


In [6]:
#block onsets
block_onsets = events[events['value'].apply(lambda x: 'Start File' in x)]
block_onsets = block_onsets.drop(columns=['duration'])
block_onsets

,onset,volume,type,value
9,10.244,6.122,Sound,14 Start File 596.wav
24,24.340,13.170,Sound,14 Start File 732.wav
38,37.380,19.690,Sound,14 Start File 689.wav
51,50.142,26.071,Sound,14 Start File 638.wav
62,61.222,31.611,Sound,14 Start File 554.wav
...,...,...,...,...
1388,1247.832,624.916,Sound,14 Start File 303.wav
1398,1254.810,628.405,Sound,14 Start File 017.wav
1408,1263.404,632.702,Sound,14 Start File 125.wav
1418,1271.672,636.836,Sound,14 Start File 392.wav


In [7]:
#sound file numbers
block_onsets['value'] = block_onsets['value'].apply(lambda x: x.split('Start File ')[1].split('.wav')[0])
block_onsets

,onset,volume,type,value
9,10.244,6.122,Sound,596
24,24.340,13.170,Sound,732
38,37.380,19.690,Sound,689
51,50.142,26.071,Sound,638
62,61.222,31.611,Sound,554
...,...,...,...,...
1388,1247.832,624.916,Sound,303
1398,1254.810,628.405,Sound,017
1408,1263.404,632.702,Sound,125
1418,1271.672,636.836,Sound,392


In [8]:
transcription = pd.DataFrame()
for i, file in enumerate(block_onsets['value']):
    block_start = block_onsets['onset'].iloc[i]
    segment = audio_onsets[audio_onsets['TextGrid'].apply(lambda x: file in x)]
    segment = segment.drop(columns=['WordCount','NumWords','PresegBegin'])
    segment['AlignOnset'] = segment['AlignOnset'] + block_start
    #concatenate all segments
    transcription = pd.concat([transcription,segment])
transcription = transcription.reset_index(drop=True)



In [9]:
transcription

,TextGrid,Transcription,AlignOnset,AlignOffset,OnsetPhoneme,OffsetPhoneme,Duration
0,596,keek,10.244,0.46,k,k,0.46
1,596,in,10.914,0.99,I,n,0.32
2,596,die,11.434,1.45,d,i,0.26
3,596,de,11.974,2.01,d,@,0.28
4,596,verbaasd,12.464,2.92,v,t,0.70
...,...,...,...,...,...,...,...
1375,322,het,1280.992,1.68,@,t,0.09
1376,322,nieuwe,1281.082,2.15,n,@,0.47
1377,322,en,1281.552,2.38,E,n,0.23
1378,322,eeuwige,1281.782,2.76,e:,@,0.38
